In [69]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [70]:
numerical = pd.read_csv('numerical.csv')
categorical = pd.read_csv('categorical.csv')
target = pd.read_csv('target.csv')
display(numerical.head(5))
display(categorical.head(5))
display(target.head(5))

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,CARDGIFT,MINRAMNT,MAXRAMNT,LASTGIFT,TIMELAG,AVGGIFT,CONTROLN,HPHONE_D,RFA_2F,CLUSTER2
0,0,60.000000,5,9,0,0,39,34,18,10,...,14,5.0,12.0,10.0,4,7.741935,95515,0,4,39
1,1,46.000000,6,9,16,0,15,55,11,6,...,1,10.0,25.0,25.0,18,15.666667,148535,0,2,1
2,1,61.611649,3,1,2,0,20,29,33,6,...,14,2.0,16.0,5.0,12,7.481481,15078,1,4,60
3,0,70.000000,1,4,2,0,23,14,31,3,...,7,2.0,11.0,10.0,9,6.812500,172556,1,4,41
4,0,78.000000,3,2,60,1,28,9,53,26,...,8,3.0,15.0,15.0,14,6.864865,7112,1,2,26


,STATE,CLUSTER,HOMEOWNR,GENDER,DATASRCE,RFA_2R,RFA_2A,GEOCODE2,DOMAIN_A,DOMAIN_B,...,DOB_YR,DOB_MM,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM
0,IL,36,H,F,3,L,E,C,T,2,...,37,12,92,8,94,2,95,12,89,11
1,CA,14,H,M,3,L,G,A,S,1,...,52,2,93,10,95,12,95,12,93,10
2,NC,43,U,M,3,L,E,C,R,2,...,0,2,91,11,92,7,95,12,90,1
3,CA,44,U,F,3,L,E,C,R,2,...,28,1,87,11,94,11,95,12,87,2
4,FL,16,H,F,3,L,F,A,S,2,...,20,1,93,10,96,1,96,1,79,3


,TARGET_B,TARGET_D
0,0,0.0
1,0,0.0
2,0,0.0
3,0,0.0
4,0,0.0


In [71]:
donors = pd.concat([numerical, categorical, target], axis=1)
donors.head(5)

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,MINRDATE_YR,MINRDATE_MM,MAXRDATE_YR,MAXRDATE_MM,LASTDATE_YR,LASTDATE_MM,FIRSTDATE_YR,FIRSTDATE_MM,TARGET_B,TARGET_D
0,0,60.000000,5,9,0,0,39,34,18,10,...,92,8,94,2,95,12,89,11,0,0.0
1,1,46.000000,6,9,16,0,15,55,11,6,...,93,10,95,12,95,12,93,10,0,0.0
2,1,61.611649,3,1,2,0,20,29,33,6,...,91,11,92,7,95,12,90,1,0,0.0
3,0,70.000000,1,4,2,0,23,14,31,3,...,87,11,94,11,95,12,87,2,0,0.0
4,0,78.000000,3,2,60,1,28,9,53,26,...,93,10,96,1,96,1,79,3,0,0.0


In [72]:
# Confirming that is there any nulls.
donors.isna().sum().sum()

0

   + Apply the Random Forests algorithm but this time only by upscaling the data to deal with the imbalance.

In [73]:
X = donors.drop(columns=['TARGET_B','TARGET_D'])
y = donors['TARGET_B']

In [74]:
print(X.shape)
print(y.shape)

(95412, 337)
(95412,)


In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [76]:
#Imbalance check
print("Imbalance Check:")
print(y_train.value_counts())

Imbalance Check:
0    72464
1     3865
Name: TARGET_B, dtype: int64


In [77]:
train_data = pd.concat([X_train, y_train], axis=1)

In [78]:
from sklearn.utils import resample
class_0 = train_data[train_data['TARGET_B'] == 0]
class_1 = train_data[train_data['TARGET_B'] == 1]

In [79]:
class_1_upsampled = resample(class_1, 
                                   replace=True,    
                                   n_samples=len(class_0))

In [80]:
upsampled_data = pd.concat([class_0, class_1_upsampled])
upsampled_data = upsampled_data.sample(frac =1)

In [81]:
upsampled_data['TARGET_B'].value_counts()

1    72464
0    72464
Name: TARGET_B, dtype: int64

In [82]:
upsampled_data.shape

(144928, 338)

In [83]:
X_train_upsampled = upsampled_data.drop('TARGET_B', axis=1)
y_train_upsampled = upsampled_data['TARGET_B']

In [84]:
# Check the number of rows
print(X_train_upsampled.shape)
print(X_test.shape)
print(y_train_upsampled.shape)
print(y_test.shape)

(144928, 337)
(19083, 337)
(144928,)
(19083,)


In [85]:
X_train_num = X_train_upsampled.select_dtypes(include = np.number)
X_test_num  = X_test.select_dtypes(include = np.number)
X_train_cat = X_train_upsampled.select_dtypes(include = object)
X_test_cat  = X_test.select_dtypes(include = object)

In [86]:
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(X_train_num) 
X_train_scaled_arr = transformer.transform(X_train_num)
X_train_scaled = pd.DataFrame(X_train_scaled_arr, columns=X_train_num.columns)
#X_train_scaled.head()
X_train_scaled.shape

(144928, 330)

In [87]:
transformer = MinMaxScaler().fit(X_test_num) 
X_test_scaled_arr = transformer.transform(X_test_num)
X_test_scaled = pd.DataFrame(X_test_scaled_arr, columns=X_test_num.columns)
#X_test_scaled.head()
X_test_scaled.shape

(19083, 330)

In [88]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first')

# Fit and Transform X_train
encoded_train_cat = encoder.fit_transform(X_train_cat).toarray()
cols = encoder.get_feature_names_out(input_features=X_train_cat.columns)
onehot_encoded_train = pd.DataFrame(encoded_train_cat, columns=cols)

# Transform X_test
encoder.set_params(handle_unknown='ignore')
encoded_test_cat = encoder.transform(X_test_cat).toarray()
onehot_encoded_test = pd.DataFrame(encoded_test_cat, columns=cols)

In [89]:
# Dataframe from X_train
X_train_treated = pd.concat([X_train_scaled, onehot_encoded_train], axis=1)
X_train_treated

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.000000,0.624862,0.666667,1.000000,0.000000,0.000000,0.212121,0.303030,0.262626,0.050505,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.515464,1.000000,1.000000,0.045643,0.090909,0.737374,0.444444,0.060606,0.151515,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.000028,0.628866,0.666667,1.000000,0.000000,0.020202,0.404040,0.333333,0.383838,0.131313,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000389,0.546392,0.833333,0.555556,0.037344,0.000000,0.242424,0.646465,0.131313,0.020202,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.000014,0.624862,0.666667,1.000000,0.000000,0.000000,0.262626,0.292929,0.383838,0.050505,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144923,0.000014,0.624862,0.666667,1.000000,0.000000,0.010101,0.393939,0.323232,0.191919,0.080808,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
144924,0.000014,0.721649,0.500000,0.888889,0.058091,0.010101,0.151515,0.404040,0.080808,0.090909,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
144925,0.000014,0.624862,0.666667,1.000000,0.000000,0.000000,0.212121,0.232323,0.000000,0.070707,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
144926,0.000000,0.624862,0.000000,1.000000,0.000000,0.000000,0.323232,0.343434,0.313131,0.070707,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [90]:
# Dataframe from X_test
X_test_treated = pd.concat([X_test_scaled, onehot_encoded_test], axis=1)
X_test_treated

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.000000,0.443299,0.333333,1.0,0.00000,0.010309,0.181818,0.303030,0.272727,0.118644,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.624862,0.333333,1.0,0.00000,0.020619,0.343434,0.323232,0.252525,0.152542,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,0.000026,0.422680,0.666667,1.0,0.00000,0.000000,0.404040,0.292929,0.383838,0.169492,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000026,0.624862,0.666667,1.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.000051,0.624862,0.166667,1.0,0.00000,0.010309,0.303030,0.262626,0.474747,0.271186,...,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19078,0.000000,0.463918,0.166667,1.0,0.00000,0.000000,0.272727,0.313131,0.191919,0.067797,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
19079,0.000000,0.628866,0.000000,1.0,0.00000,0.010309,0.515152,0.303030,0.202020,0.016949,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
19080,0.000000,0.340206,0.166667,1.0,0.00000,0.000000,0.323232,0.424242,0.232323,0.271186,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
19081,0.000026,0.731959,0.500000,1.0,0.06639,0.020619,0.323232,0.131313,0.434343,0.050847,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [91]:
#X_train_upsampled.head()
#X_train_upsampled.dtypes

In [92]:
y_train_upsampled.head()

38521    1
84368    1
17557    0
44361    1
91586    0
Name: TARGET_B, dtype: int64

In [93]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

clf = RandomForestClassifier(max_depth=5, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=42)
clf.fit(X_train_treated, y_train_upsampled)
print(clf.score(X_train_treated, y_train_upsampled))
print(clf.score(X_test_treated, y_test))


0.6203218149701921
0.655924120945344


In [94]:
y_pred = clf.predict(X_test_treated)
display(y_test.value_counts())
display(confusion_matrix(y_test, y_pred))

0    18105
1      978
Name: TARGET_B, dtype: int64

array([[12017,  6088],
       [  478,   500]], dtype=int64)

   + Use Feature Selections that you have learned in class to decide if you want to use all of the features (Variance Threshold, RFE, PCA, etc.)

In [95]:
X_train_treated.reset_index(drop=True, inplace=True)
y_train_upsampled.reset_index(drop=True, inplace=True)
X_test_treated.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [96]:
data_train = pd.concat([X_train_treated, y_train_upsampled], axis=1)
data_train.head(5)
#print(data_train.isna().sum().sum())

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
0,0.000000,0.624862,0.666667,1.000000,0.000000,0.000000,0.212121,0.303030,0.262626,0.050505,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.000000,0.515464,1.000000,1.000000,0.045643,0.090909,0.737374,0.444444,0.060606,0.151515,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
2,0.000028,0.628866,0.666667,1.000000,0.000000,0.020202,0.404040,0.333333,0.383838,0.131313,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.000389,0.546392,0.833333,0.555556,0.037344,0.000000,0.242424,0.646465,0.131313,0.020202,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
4,0.000014,0.624862,0.666667,1.000000,0.000000,0.000000,0.262626,0.292929,0.383838,0.050505,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0


In [97]:
data_test = pd.concat([X_test_treated, y_test], axis=1)
data_test.head(5)
#print(data_test.isna().sum().sum())

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
0,0.000000,0.443299,0.333333,1.0,0.0,0.010309,0.181818,0.303030,0.272727,0.118644,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
1,0.000000,0.624862,0.333333,1.0,0.0,0.020619,0.343434,0.323232,0.252525,0.152542,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
2,0.000026,0.422680,0.666667,1.0,0.0,0.000000,0.404040,0.292929,0.383838,0.169492,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.000026,0.624862,0.666667,1.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
4,0.000051,0.624862,0.166667,1.0,0.0,0.010309,0.303030,0.262626,0.474747,0.271186,...,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0


In [98]:
data_fea = pd.concat([data_train,data_test], axis=0)
data_fea

,TCODE,AGE,INCOME,WEALTH1,HIT,MALEMILI,MALEVET,VIETVETS,WWIIVETS,LOCALGOV,...,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,TARGET_B
0,0.000000,0.624862,0.666667,1.000000,0.000000,0.000000,0.212121,0.303030,0.262626,0.050505,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.000000,0.515464,1.000000,1.000000,0.045643,0.090909,0.737374,0.444444,0.060606,0.151515,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
2,0.000028,0.628866,0.666667,1.000000,0.000000,0.020202,0.404040,0.333333,0.383838,0.131313,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.000389,0.546392,0.833333,0.555556,0.037344,0.000000,0.242424,0.646465,0.131313,0.020202,...,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1
4,0.000014,0.624862,0.666667,1.000000,0.000000,0.000000,0.262626,0.292929,0.383838,0.050505,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19078,0.000000,0.463918,0.166667,1.000000,0.000000,0.000000,0.272727,0.313131,0.191919,0.067797,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
19079,0.000000,0.628866,0.000000,1.000000,0.000000,0.010309,0.515152,0.303030,0.202020,0.016949,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1
19080,0.000000,0.340206,0.166667,1.000000,0.000000,0.000000,0.323232,0.424242,0.232323,0.271186,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
19081,0.000026,0.731959,0.500000,1.000000,0.066390,0.020619,0.323232,0.131313,0.434343,0.050847,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [99]:
data_fea.isna().sum().sum()

0

In [100]:
X_data_fea = data_fea.drop('TARGET_B', axis=1)
y_data_fea = data_fea['TARGET_B']

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

logreg = LogisticRegression(max_iter=10)
rfe = RFE(logreg, n_features_to_select=20, verbose=False)
rfe.fit(X_data_fea, y_data_fea)

In [ ]:
rfe.ranking_

In [ ]:
df = pd.DataFrame(data = rfe.ranking_, columns=['Rank'])
df['Column_name'] = pd.DataFrame(X_data_fea).columns
df

In [ ]:
selected_features = df[df['Rank']==1]
selected_features
#len(selected_features)

In [ ]:
selected_features = selected_features['Column_name']
col_rfe = data_fea[selected_features]
col_rfe

   + Re-run the Random Forest algorithm to determine if the Feature Selection has improved the results.

In [ ]:
X_col_rfe = col_rfe
y_col_rfe = data_fea['TARGET_B']

In [ ]:
X_train_rfe, X_test_rfe, y_train_rfe, y_test_rfe = train_test_split(X_col_rfe, y_col_rfe, test_size=0.2)

In [ ]:
clf_rfe = RandomForestClassifier(max_depth=5, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=42)
clf_rfe.fit(X_train_rfe, y_train_rfe)
print(clf_rfe.score(X_train_rfe, y_train_rfe))
print(clf_rfe.score(X_test_rfe, y_test_rfe))


# VarianceThreshold

In [101]:
from sklearn.feature_selection import VarianceThreshold

In [102]:
X_data_fea = data_fea.drop('TARGET_B', axis=1)
y_data_fea = data_fea['TARGET_B']

In [103]:
var_threshold = 0.02
sel = VarianceThreshold(threshold=(var_threshold))

In [104]:
sel = sel.fit(X_data_fea)

In [105]:
sel.get_support()
var_list=list(sel.get_support())
droplist_var=[col[0] for col in zip(X_data_fea.columns, var_list) if col[1] == False]
print(droplist_var)
len(droplist_var)

['TCODE', 'HIT', 'MALEMILI', 'MALEVET', 'LOCALGOV', 'STATEGOV', 'FEDGOV', 'POP901', 'POP902', 'POP903', 'POP90C4', 'POP90C5', 'ETH3', 'ETH4', 'ETH5', 'ETH6', 'ETH7', 'ETH8', 'ETH9', 'ETH10', 'ETH11', 'ETH12', 'ETH13', 'ETH14', 'ETH15', 'ETH16', 'AGE901', 'AGE902', 'AGE903', 'AGE904', 'AGE905', 'AGE906', 'AGE907', 'CHIL1', 'CHIL2', 'CHIL3', 'AGEC1', 'AGEC2', 'AGEC3', 'AGEC4', 'AGEC5', 'AGEC6', 'AGEC7', 'CHILC1', 'CHILC2', 'CHILC3', 'CHILC4', 'CHILC5', 'HHAGE1', 'HHAGE2', 'HHAGE3', 'HHN1', 'HHN2', 'HHN4', 'HHN5', 'HHN6', 'MARR1', 'MARR2', 'MARR3', 'MARR4', 'HHP1', 'HHP2', 'DW3', 'DW7', 'DW8', 'DW9', 'HU3', 'HU4', 'HHD1', 'HHD4', 'HHD5', 'HHD6', 'HHD7', 'HHD8', 'HHD9', 'HHD10', 'HHD11', 'HHD12', 'ETHC1', 'ETHC2', 'ETHC3', 'ETHC4', 'ETHC5', 'ETHC6', 'HUR1', 'RHP1', 'RHP2', 'RHP3', 'RHP4', 'HUPA1', 'HUPA4', 'HUPA5', 'HUPA7', 'DMA', 'IC1', 'IC2', 'IC3', 'IC4', 'IC5', 'IC7', 'IC8', 'IC9', 'IC10', 'IC11', 'IC12', 'IC13', 'IC14', 'IC15', 'IC16', 'IC17', 'IC18', 'IC19', 'IC20', 'IC21', 'IC22', '

241

In [106]:
X_data_fea_1 = X_data_fea.drop(droplist_var, axis=1)
X_data_fea_1

,AGE,INCOME,WEALTH1,VIETVETS,WWIIVETS,WEALTH2,POP90C1,POP90C2,POP90C3,ETH1,...,RFA_2A_E,RFA_2A_F,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U
0,0.624862,0.666667,1.000000,0.303030,0.262626,1.000000,0.000000,0.979798,0.030303,0.929293,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.515464,1.000000,1.000000,0.444444,0.060606,1.000000,0.000000,0.000000,1.000000,0.939394,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.628866,0.666667,1.000000,0.333333,0.383838,1.000000,1.000000,0.000000,0.000000,0.858586,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.546392,0.833333,0.555556,0.646465,0.131313,0.555556,1.000000,0.000000,0.000000,0.949495,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.624862,0.666667,1.000000,0.292929,0.383838,0.000000,0.000000,0.000000,1.000000,0.989899,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19078,0.463918,0.166667,1.000000,0.313131,0.191919,0.444444,0.191919,0.000000,0.818182,0.979798,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
19079,0.628866,0.000000,1.000000,0.303030,0.202020,0.888889,0.000000,0.000000,1.000000,0.989899,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
19080,0.340206,0.166667,1.000000,0.424242,0.232323,1.000000,1.000000,0.000000,0.000000,0.878788,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
19081,0.731959,0.500000,1.000000,0.131313,0.434343,1.000000,1.000000,0.000000,0.000000,0.151515,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [107]:
len(data_fea['TARGET_B'])

164011

# Ramdonclassifer

In [108]:
X_var = X_data_fea_1
y_var = data_fea['TARGET_B']

In [109]:
X_train_var, X_test_var, y_train_var, y_test_var = train_test_split(X_var, y_var, test_size=0.2)

In [110]:
clf_rfe = RandomForestClassifier(max_depth=10, 
                             min_samples_split=20, 
                             min_samples_leaf =20, 
                             max_samples=0.8)
clf_rfe.fit(X_train_var, y_train_var)
print(clf_rfe.score(X_train_var, y_train_var))
print(clf_rfe.score(X_test_var, y_test_var))

0.8030150600573136
0.7856293631680029


   * Discuss the output and its impact in the business scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the business?

In [111]:
predictions = clf_rfe.predict(X_test_var)
predictions

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [112]:
from sklearn.metrics import confusion_matrix

In [113]:
array = confusion_matrix(y_test_var, predictions)
print('             Predicted Labels')
print('             |   A    |     B')
print('---------------------------------')
print('True label A | ',array[0][0],'|   ', array[0][1])
print('---------------------------------')
print('           B | ',array[1][0],' |   ', array[1][1])


             Predicted Labels
             |   A    |     B
---------------------------------
True label A |  16438 |    1644
---------------------------------
           B |  5388  |    9333


In [114]:
predictions_1 = clf_rfe.predict(X_var)
predictions_1

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

In [118]:
len(predictions_1)

164011

In [120]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_var, predictions_1)
precision = precision_score(y_var, predictions_1)
recall = recall_score(y_var, predictions_1)
f1 = f1_score(y_var, predictions_1)

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1 score:', f1)

Accuracy: 0.7995378358768619
Precision: 0.8687234119914191
Recall: 0.6506494921162278
F1 score: 0.7440364972595914


In [116]:
X_data_fea_1['PREDICTIONS']= list(predictions_1)
X_data_fea_2 = pd.concat([X_data_fea_1, data_fea['TARGET_B']], axis=1)
X_data_fea_2

,AGE,INCOME,WEALTH1,VIETVETS,WWIIVETS,WEALTH2,POP90C1,POP90C2,POP90C3,ETH1,...,RFA_2A_G,GEOCODE2_B,GEOCODE2_C,GEOCODE2_D,DOMAIN_A_R,DOMAIN_A_S,DOMAIN_A_T,DOMAIN_A_U,PREDICTIONS,TARGET_B
0,0.624862,0.666667,1.000000,0.303030,0.262626,1.000000,0.000000,0.979798,0.030303,0.929293,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.515464,1.000000,1.000000,0.444444,0.060606,1.000000,0.000000,0.000000,1.000000,0.939394,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,1
2,0.628866,0.666667,1.000000,0.333333,0.383838,1.000000,1.000000,0.000000,0.000000,0.858586,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0
3,0.546392,0.833333,0.555556,0.646465,0.131313,0.555556,1.000000,0.000000,0.000000,0.949495,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1
4,0.624862,0.666667,1.000000,0.292929,0.383838,0.000000,0.000000,0.000000,1.000000,0.989899,...,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19078,0.463918,0.166667,1.000000,0.313131,0.191919,0.444444,0.191919,0.000000,0.818182,0.979798,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0
19079,0.628866,0.000000,1.000000,0.303030,0.202020,0.888889,0.000000,0.000000,1.000000,0.989899,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1,1
19080,0.340206,0.166667,1.000000,0.424242,0.232323,1.000000,1.000000,0.000000,0.000000,0.878788,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0
19081,0.731959,0.500000,1.000000,0.131313,0.434343,1.000000,1.000000,0.000000,0.000000,0.151515,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0,0


In [117]:
X_data_fea_2.to_csv('donors_with_targetB_predictions.csv', index=False)